# Chapter 8: Time Series Analysis

In finance and economics, a huge amount of our data is in the format of time-series,
such as stock prices and Gross Domestic Products (GDP). From Chapter 4, Sources
of Data, it is shown that from Yahoo!Finance, we could download daily, weekly, and
monthly historical price time-series. From Federal Reserve Bank's Economics Data
Library (FRED), we could retrieve many historical time-series such as GDP. For
time-series, there exist many issues, such as how to estimate returns from historical
price data, how to merge datasets with the same or different frequencies, seasonality,
and detect auto-correlation. Understanding those properties is vitally important for
our knowledge development.

In this chapter, the following topics will be covered:
• Introduction to time-series analysis

• Design a good date variable, and merging different datasets by date

• Normal distribution and normality test

• Term structure of interest rates, 52-week high, and low trading strategy

• Return estimation and converting daily returns to monthly or annual returns

• T-test, F-test, and Durbin-Watson test for autocorrelation

• Fama-MacBeth regression

• Roll (1984) spread, Amihud's (2002) illiquidity, and Pastor and Stambaugh's
(2003) liquidity measure

• January effect and weekday effect

• Retrieving high-frequency data from Google Finance and Prof. Hasbrouck's
TORQ database (Trade, Order, Report, and Quotation)

• Introduction to CRSP (Center for Research in Security Prices) database

## Introduction to Time Series

Most finance data is in the format of time-series, see the following several examples.
The first one shows how to download historical, daily stock price data from
Yahoo!Finance for a given ticker's beginning and ending dates:

In [340]:
import yfinance as yf
x = yf.download('IBM', start="2016-01-01", end="2016-01-21", rounding=True)
print(x)

[*********************100%***********************]  1 of 1 completed
              Open    High     Low   Close  Adj Close    Volume
Date                                                           
2016-01-04  129.64  129.99  128.34  129.97      89.53   5469952
2016-01-05  130.75  130.87  128.92  129.88      89.47   4105341
2016-01-06  128.47  129.62  127.74  129.23      89.02   4509201
2016-01-07  127.82  129.08  126.61  127.02      87.50   7348987
2016-01-08  127.32  127.93  125.54  125.84      86.69   4981784
2016-01-11  126.01  127.93  125.97  127.37      87.74   5203222
2016-01-12  127.77  127.90  125.36  127.06      87.53   5312320
2016-01-13  127.63  128.37  125.33  125.40      86.39   4916305
2016-01-14  126.13  127.91  125.45  127.07      87.53   5972242
2016-01-15  124.29  125.12  123.20  124.31      85.64   9422891
2016-01-19  124.39  126.29  122.43  122.48      84.37  10438662
2016-01-20  113.25  118.54  112.81  116.50      80.25  16901059


Let's see if webreader still works. EDIT: IT doesn't for yahoo. Not an issue. Now let's grab some GDP data.

In [341]:
import pandas_datareader.data as web
import datetime
begdate = datetime.datetime(1900, 1, 1)
enddate = datetime.datetime(2017, 1, 27)
y= web.DataReader("GDP", "fred", begdate,enddate)
print(x.head(2))
print(x.tail(3))

              Open    High     Low   Close  Adj Close   Volume
Date                                                          
2016-01-04  129.64  129.99  128.34  129.97      89.53  5469952
2016-01-05  130.75  130.87  128.92  129.88      89.47  4105341
              Open    High     Low   Close  Adj Close    Volume
Date                                                           
2016-01-15  124.29  125.12  123.20  124.31      85.64   9422891
2016-01-19  124.39  126.29  122.43  122.48      84.37  10438662
2016-01-20  113.25  118.54  112.81  116.50      80.25  16901059


GDP data works fine when read through the pandas_datareader module.

## Merging datasets based on a date variable

To make our time-series more manageable, it is a great idea to generate a date
variable. When talking about such a variable, readers could think about year (YYYY),
year and month (YYYYMM) or year, month, and day (YYYYMMDD). For just the
year, month, and day combination, we could have many forms. Using January 20,
2017 as an example, we could have 2017-1-20, 1/20/2017, 20Jan2017, 20-1-2017, and
the like. In a sense, a true date variable, in our mind, could be easily manipulated.
Usually, the true date variable takes a form of year-month-day or other forms of its
variants. Assume the date variable has a value of 2000-12-31. After adding one day to
its value, the result should be 2001-1-1.

We could easily use the pandas.date_range() function to generate our time-series;
refer to the following example:

In [342]:
import pandas as pd
import numpy as np
import scipy as sp

np.random.seed(1257)
mean=0.10
std=0.2
ddate = pd.date_range('1/1/2016', periods=252)
n=len(ddate)
rets= np.random.normal(mean,std,n)
data = pd.DataFrame(rets, index=ddate,columns=['RET'])
print(data.head())

                 RET
2016-01-01  0.431031
2016-01-02  0.279193
2016-01-03  0.002549
2016-01-04  0.109546
2016-01-05  0.068252


In [343]:
print(x[0:4])

              Open    High     Low   Close  Adj Close   Volume
Date                                                          
2016-01-04  129.64  129.99  128.34  129.97      89.53  5469952
2016-01-05  130.75  130.87  128.92  129.88      89.47  4105341
2016-01-06  128.47  129.62  127.74  129.23      89.02  4509201
2016-01-07  127.82  129.08  126.61  127.02      87.50  7348987


Here are several ways to define a date variable:

In [344]:
# For a range of dates
pd.date_range('1/1/2017',periods=252)

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03',
               '2017-09-04', '2017-09-05', '2017-09-06', '2017-09-07',
               '2017-09-08', '2017-09-09'],
              dtype='datetime64[ns]', length=252, freq='D')

In [345]:
# For a single date
datetime.date(2017,1,20)

datetime.date(2017, 1, 20)

In [346]:
# For today's date
datetime.date.today()

datetime.date(2024, 2, 19)

In [347]:
# Get the current time
from datetime import datetime
datetime.now()


datetime.datetime(2024, 2, 19, 15, 23, 44, 205778)

Retrieving the year, month, and day from a date variable is used quite frequently
when dealing with time-series—see the following Python program by using the
strftime() function. The corresponding output is in the following right panel. The
format of those results of year, month, and day, is string:

In [348]:
import datetime
today=datetime.date.today()
year=today.strftime("%Y")
year2=today.strftime("%y")
month=today.strftime("%m")
day=today.strftime("%d")
print(year,month,day,year2)

2024 02 19 24


## Return Estimation

With price data, we could calculate returns. In addition, sometimes we have to
convert daily returns to weekly or monthly, or convert monthly returns to quarterly
or annual ones. Thus, understanding how to estimate returns and their conversion is
vital. Assume that we have the following four prices:

In [349]:
p=[1,1.1,0.9,1.05]

In [350]:
print(p[:-1])
print(p[1:])

[1, 1.1, 0.9]
[1.1, 0.9, 1.05]


To estimate returns, we could use the following code:

Alternatively we can use numpy and the pandas libraries to do the same calculation, but in dataframe form.

In [351]:
p=[1,1.1,0.9,1.05]
a=pd.DataFrame({'Price':p})
a['Ret']=a['Price'].diff()/a['Price'].shift(1)
print(a)

   Price       Ret
0   1.00       NaN
1   1.10  0.100000
2   0.90 -0.181818
3   1.05  0.166667


Let's now analyze daily price data using what we've done above.

In [352]:
x = yf.download('IBM', start="2013-01-01", end="2013-11-9", rounding=True)
ret=x['Close'].pct_change()
ret

[*********************100%***********************]  1 of 1 completed


Date
2013-01-02         NaN
2013-01-03   -0.005540
2013-01-04   -0.006535
2013-01-07   -0.004368
2013-01-08   -0.001408
                ...   
2013-11-04    0.005778
2013-11-05   -0.013404
2013-11-06    0.007528
2013-11-07    0.004495
2013-11-08   -0.000058
Name: Close, Length: 217, dtype: float64

In [353]:
x.index[0:3]

DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04'], dtype='datetime64[ns]', name='Date', freq=None)

In [354]:
x['Close'][0:3] 

Date
2013-01-02    187.72
2013-01-03    186.68
2013-01-04    185.46
Name: Close, dtype: float64

In [355]:
ret[0:3]

Date
2013-01-02         NaN
2013-01-03   -0.005540
2013-01-04   -0.006535
Name: Close, dtype: float64

## Converting daily returns to monthly ones

Sometimes, we need to convert daily returns to monthly or annual ones. Here is our
procedure. First, we estimate the daily log returns. We then take a summation of
all daily log returns within each month to find out the corresponding monthly log
returns. The final step is to convert a log monthly return to a monthly percentage
return. Assume that we have the price data of p0, p1, p2, …., p20, where p0 is the last
trading price of the last month, p1 is the first price of this month, and p20 is the last
price of this month. Thus, this month's percentage return is given as follows:

R(monthly) = (p20 - p0) / p0

LogR(monthly) = log (p20/p0)

R(monthly) = exp(LogR) -1

In [356]:
logret = pd.DataFrame(np.log(1+ x['Close'].pct_change()))
logret = logret.rename(columns={'Close':'Returns'})
yyyymm=[]
d0=x.index

logret
d0


DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07',
               '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11',
               '2013-01-14', '2013-01-15',
               ...
               '2013-10-28', '2013-10-29', '2013-10-30', '2013-10-31',
               '2013-11-01', '2013-11-04', '2013-11-05', '2013-11-06',
               '2013-11-07', '2013-11-08'],
              dtype='datetime64[ns]', name='Date', length=217, freq=None)

In [357]:
#
logret['Months'] = d0.strftime("%Y-%m")
logret

retMonthly=logret.groupby(logret['Months']).sum()
print(retMonthly.head())


          Returns
Months           
2013-01  0.033628
2013-02 -0.011084
2013-03  0.060232
2013-04 -0.051779
2013-05  0.026702


## Merging datasets by date

The following program merges the daily adjusted closing price of IBM with the
daily Fama-French 3-factor time-series.

In [444]:
ticker='IBM'
begdate=datetime.date(2016,1,2)
enddate=datetime.date(2017,1,9)
x = yf.download(ticker, start = begdate, end = enddate, rounding = True)

f3factor = pd.read_csv('/opt/code/test_repo/Python-For-Finance-2nd-Edition/F-F_Research_Data_Factors_daily.CSV').dropna()
f3factor['Date'] = pd.to_numeric(f3factor['Date'], errors="coerce").astype(int).dropna()
f3factor['Date'] = pd.to_datetime(f3factor['Date'], format='%Y%m%d')
f3factor = f3factor.set_index('Date')

myName=ticker+'_adjClose'
x2= pd.DataFrame(x['Close']).rename(columns={'Close':myName})

f3factor


[*********************100%***********************]  1 of 1 completed


,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009
...,...,...,...,...
2023-12-22,0.21,0.64,0.09,0.021
2023-12-26,0.48,0.69,0.46,0.021
2023-12-27,0.16,0.14,0.12,0.021


In [359]:
final=pd.merge(x2,f3factor,left_index=True,right_index=True)
print(final.head())
final.head()

            IBM_adjClose  Mkt-RF   SMB   HML   RF
Date                                             
2016-01-04        129.97   -1.59 -0.87  0.52  0.0
2016-01-05        129.88    0.12 -0.19  0.01  0.0
2016-01-06        129.23   -1.35 -0.14  0.00  0.0
2016-01-07        127.02   -2.44 -0.29  0.08  0.0
2016-01-08        125.84   -1.11 -0.50 -0.03  0.0


,IBM_adjClose,Mkt-RF,SMB,HML,RF
Date,,,,,
2016-01-04,129.97,-1.59,-0.87,0.52,0.0
2016-01-05,129.88,0.12,-0.19,0.01,0.0
2016-01-06,129.23,-1.35,-0.14,0.00,0.0
2016-01-07,127.02,-2.44,-0.29,0.08,0.0
2016-01-08,125.84,-1.11,-0.50,-0.03,0.0


## Understanding the interpolation technique

Interpolation is a technique used quite frequently in finance. In the following
example, we have to replace two missing values, NaN, between 2 and 6. The
pandas.interpolate() function, for a linear interpolation, is used to fill in the
two missing values:

In [360]:
import pandas as pd
import numpy as np
nn=np.nan
x=pd.Series([1,2,nn,nn,6])
print(x.interpolate())

0    1.000000
1    2.000000
2    3.333333
3    4.666667
4    6.000000
dtype: float64


## Merging data with different frequencies

The following Python program merges two datasets: US Gross Domestic Product
(GDP) data with a quarterly frequency and ffMonthly, http://canisius.
edu/~yany/python/ffMonthly.pkl with a monthly frequency.


The interpolation methodology discussed previously is applied to the missing
months in terms of GDP data. The ffMonthly dataset is assumed to be saved
in the c:/temp/ directory:

In [361]:
#get Fama French factors
import getFamaFrenchFactors as gff

ff3 = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
ff3 = ff3.rename(columns={'date_ff_factors':'Date'})
ff3['Date'] = ff3['Date'].dt.strftime('%Y-%m')
ff3 = ff3.set_index('Date')
ff3



,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07,0.0296,-0.0256,-0.0243,0.0022
1926-08,0.0264,-0.0117,0.0382,0.0025
1926-09,0.0036,-0.0140,0.0013,0.0023
1926-10,-0.0324,-0.0009,0.0070,0.0032
1926-11,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2023-08,-0.0239,-0.0316,-0.0106,0.0045
2023-09,-0.0524,-0.0251,0.0152,0.0043
2023-10,-0.0319,-0.0387,0.0019,0.0047


In [362]:
begdate = datetime.datetime(1900, 1, 1)
enddate = datetime.datetime(2024, 1, 27)
GDP = web.DataReader("GDP", "fred", begdate, enddate)
GDP.index = GDP.index.strftime('%Y-%m')
GDP

,GDP
DATE,
1947-01,243.164
1947-04,245.968
1947-07,249.585
1947-10,259.745
1948-01,265.742
...,...
2022-10,26408.405
2023-01,26813.601
2023-04,27063.012


In [363]:
final=pd.merge(ff3,GDP,left_index=True,right_index=True,how='left')
tt=final['GDP']
GDP2=pd.Series(tt).interpolate()
final['GDP2']=GDP2

In [364]:
final.tail(20)

,Mkt-RF,SMB,HML,RF,GDP,GDP2
Date,,,,,,
2022-05,-0.0034,-0.0183,0.0839,0.0003,NaN,25694.395000
2022-06,-0.0843,0.0210,-0.0597,0.0006,NaN,25844.517000
2022-07,0.0957,0.0281,-0.0405,0.0008,25994.639,25994.639000
2022-08,-0.0377,0.0140,0.0029,0.0019,NaN,26132.561000
2022-09,-0.0935,-0.0081,0.0005,0.0019,NaN,26270.483000
2022-10,0.0783,0.0006,0.0801,0.0023,26408.405,26408.405000
2022-11,0.0460,-0.0352,0.0138,0.0029,NaN,26543.470333
2022-12,-0.0641,-0.0069,0.0137,0.0033,NaN,26678.535667
2023-01,0.0665,0.0501,-0.0401,0.0035,26813.601,26813.601000


## Tests of normality

In finance, knowledge about normal distribution is very important for two reasons.
First, stock returns are assumed to follow a normal distribution. Second, the error
terms from a good econometric model should follow a normal distribution with a
zero mean. However, in the real world, this might not be true for stocks. On the other
hand, whether stocks or portfolios follow a normal distribution could be tested by
various so-called normality tests. The Shapiro-Wilk test is one of them. For the first
example, random numbers are drawn from a normal distribution. As a consequence,
the test should confirm that those observations follow a normal distribution:

In [365]:
from scipy import stats
np.random.seed(12345)
mean=0.1
std=0.2
n=5000
ret=np.random.normal(loc=0,scale=std,size=n)
print('W-test, and P-value')
print(stats.shapiro(ret))

W-test, and P-value
ShapiroResult(statistic=0.9995947480201721, pvalue=0.40313461422920227)


Assume that our confidence level is 95%, that is, alpha=0.05. The first value of the
result is the test statistic, and the second one is its corresponding P-value. Since
the P-value is so big, much bigger than 0.05, we accept the null hypothesis that the
returns follow a normal distribution. For the second example, random numbers are
drawn from a uniform distribution:

In [366]:
np.random.seed(12345)
n=5000
ret=np.random.uniform(size=n)
print('W-test, and P-value')
print(stats.shapiro(ret))

W-test, and P-value
ShapiroResult(statistic=0.9537610411643982, pvalue=4.076294275851805e-37)


Since the P-value is close to zero, we reject the null hypothesis. In other words, those
observations do not follow a normal distribution. The third example verifies whether
IBM's returns follow a normal distribution. The last five year's daily data from
Yahoo! Finance is used for the test. The last five year's daily data from
Yahoo! Finance is used for the test. The null hypothesis is that IBM's daily returns are
drawn from a normal distribution:

In [367]:
x = yf.download('IBM', start="2019-01-01", end="2024-01-21", rounding=True)
ret = x['Close'].pct_change().dropna()

print("Ticker =",ticker, "W-test, and P-value")
print(stats.shapiro(ret))

[*********************100%***********************]  1 of 1 completed
Ticker = IBM W-test, and P-value
ShapiroResult(statistic=0.8869530558586121, pvalue=1.762584302639909e-29)


Since this P-value is so close to zero, we reject the null hypothesis. In other
words, we conclude that IBM's daily returns do not follow a normal distribution.
For a normality test, we could also apply the Anderson-Darling test, which is a
modification of the Kolmogorov-Smirnov test, to verify whether the observations
follow a particular distribution. See the following code:

In [368]:
print( stats.anderson(ret) )

AndersonResult(statistic=26.0722603874824, critical_values=array([0.574, 0.654, 0.785, 0.915, 1.089]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.0004869297522596514, scale=0.016585637480049348)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


Here, we have three sets of values: the Anderson-Darling test statistic, a set of critical
values, and a set of corresponding confidence levels, such as 15 percent, 10 percent,
5 percent, 2.5 percent, and 1 percent, as shown in the previous output. If we choose
a 1 percent confidence level—the last value of the third set—the critical value is
1.089, the last value of the second set. Since our testing statistic is 26, which is
much higher than the critical value of 1.089, we reject the null hypothesis. Thus, our
Anderson-Darling test leads to the same conclusion as our Shapiro-Wilk test. One
of the beauties of the scipy.stats.anderson() test is that we can test for other
distributions. After applying the help() function, we would get the following list. 

In [369]:
help(stats.anderson)

Help on function anderson in module scipy.stats._morestats:

anderson(x, dist='norm')
    Anderson-Darling test for data coming from a particular distribution.
    
    The Anderson-Darling test tests the null hypothesis that a sample is
    drawn from a population that follows a particular distribution.
    For the Anderson-Darling test, the critical values depend on
    which distribution is being tested against.  This function works
    for normal, exponential, logistic, or Gumbel (Extreme Value
    Type I) distributions.
    
    Parameters
    ----------
    x : array_like
        Array of sample data.
    dist : {'norm', 'expon', 'logistic', 'gumbel', 'gumbel_l', 'gumbel_r', 'extreme1'}, optional
        The type of distribution to test against.  The default is 'norm'.
        The names 'extreme1', 'gumbel_l' and 'gumbel' are synonyms for the
        same distribution.
    
    Returns
    -------
    result : AndersonResult
        An object with the following attributes:
    
 

## Estimating fat tails
One of the important properties of a normal distribution is that we could use mean
and standard deviation, the first two moments, to fully define the whole distribution.
For n returns of a security, its first four moments are defined in equation (1). Things to pay attention to include the variance, standard deviation, skewness, and kurtosis. The kurtosis reflects the impact of extreme values because of its power of four. 

The following code outputs these four variables:


In [370]:
import numpy as np
from numpy import random
np.random.seed(12345)
ret = random.normal(0,1,500000)
print('mean =', np.mean(ret))
print('std =',np.std(ret))
print('skewness=',stats.skew(ret))
print('kurtosis=',stats.kurtosis(ret))
import numpy as np

mean = 0.000822665171471418
std = 0.9989895658655303
skewness= 0.006299066118437377
kurtosis= 0.0015439776051819898


The mean, skewness, and kurtosis are all close to zero, while the standard deviation
is close to one. Next, we estimate the four moments for S&P500 based on its daily
returns as follows:

In [371]:
ticker="^GSPC"
begdate=datetime.date(1926,1,1)
enddate=datetime.date(2016,12,31)

p = yf.download(ticker, start=begdate, end=enddate, rounding=True)
ret = p['Close'].pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


In [372]:
print( 'S&P500 n =',len(ret))
print( 'S&P500 mean =',round(np.mean(ret),8))
print('S&P500 std =',round(np.std(ret),8))
print('S&P500 skewness=',round(stats.skew(ret),8))
print('S&P500 kurtosis=',round(stats.kurtosis(ret),8))

S&P500 n = 22354
S&P500 mean = 0.0002882
S&P500 std = 0.01195134
S&P500 skewness= -0.08475558
S&P500 kurtosis= 17.42919902


From this we can see that the SP500 index is skewed towards the left and leptokurtic (kurtosis > 3), ie fat tailed.

## T-test and F-test

In finance, a T-test could be viewed as one of the most widely used statistical
hypothesis tests in which the test statistic follows a student's t distribution if the null
hypothesis is supported. We know that the mean for a standard normal distribution
is zero. In the following program, we generate 1,000 random numbers from a
standard normal distribution. Then, we conduct two tests: test whether the mean is
0.5, and test whether the mean is zero:

In [373]:
from scipy import stats
import numpy as np

np.random.seed(1235)
x = stats.norm.rvs(size = 10000)
print("T-value P-value (two-tail)")
print(stats.ttest_1samp(x,0.5))
print(stats.ttest_1samp(x,0))

T-value P-value (two-tail)
TtestResult(statistic=-49.76347123142897, pvalue=0.0, df=9999)
TtestResult(statistic=-0.26310321925083024, pvalue=0.7924764437516486, df=9999)


For the first test, in which we test whether the time-series has a mean of 0.5, we reject
the null hypothesis since the T-value is 49.76 and the P-value is 0. For the second test,
we accept the null hypothesis since the T-value is close to -0.26 and the P-value is
0.79. In the following program, we test whether the mean of the daily returns from
IBM in 2013 is zero:

In [374]:
ticker = "IBM"
begdate = datetime.date(2013,1,1)
enddate = datetime.date(2013,12,31)
p = yf.download(ticker, start = begdate, end = enddate, rounding = True)
ret = p['Close'].pct_change().dropna()

print(' Mean T-value P-value ' )
print(round(np.mean(ret),5), stats.ttest_1samp(ret,0))

[*********************100%***********************]  1 of 1 completed
 Mean T-value P-value 
-0.00014 TtestResult(statistic=-0.1875560297924851, pvalue=0.8513774412281602, df=249)


The T-value is -.18 and the p-value is 0.85 - indicating that our results are not different (statistically significantly) from the null hypothesis. Thus, we accept the null hypothesis, that is, the daily mean return is statistically the same as zero.

## Tests of equal variances

Next, we test whether two variances for IBM and DELL are the same or not over a
five-year period from 2012 to 2016. The function called sp.stats.bartlet() performs
Bartlett's test for equal variances with a null hypothesis that all input samples are from
populations with equal variances. The outputs are the T-value and P-value:

In [375]:
begdate = datetime.date(2012,1,1)
enddate = datetime.date(2016,12,31)

def ret_f(ticker,begdate,enddate):
    p = yf.download(ticker,start = begdate, end = enddate, rounding = True)
    return p['Adj Close'].pct_change().dropna()

y=ret_f('IBM',begdate,enddate)
x=ret_f('DELL',begdate,enddate)
print(sp.stats.bartlett(x,y))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
BartlettResult(statistic=16.59601967571595, pvalue=4.6247980495427416e-05)


With a test statistic of 16.59 and a p-value extremely close to 0, we can conclude that these two stocks will have different variances for their daily stock returns from 2012 to 2015 for any significance level.

## Testing the January effect

In this section, we use IBM's data to test the existence of the so-called January effect,
which states that stock returns in January are statistically different from those in
other months. First, we collect the daily price for IBM from Yahoo! Finance. Then, we
convert daily returns to monthly ones. After that, we classify all monthly returns into
two groups: returns in January versus returns in other months.

Finally, we test the equality of group means as shown in the following code:

In [376]:
import pandas as pd
begdate=datetime.date(1962,1,1)
enddate=datetime.date(2016,12,31)

x = yf.download(ticker, start = begdate, end = enddate, rounding = True)
logret = pd.DataFrame(np.log(1+ x['Adj Close'].pct_change()))
logret = logret.rename(columns={'Adj Close':'Log Return'})
results = pd.merge(x,logret,left_index=True,right_index=True)
results['Month'] = results.index.strftime('%Y-%m')

retM=results.groupby(results['Month']).sum()
retM


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Log Return
Month,,,,,,,
1962-01,155.95,156.87,154.77,155.37,32.86,9264945,-0.053346
1962-02,132.41,133.03,131.78,132.26,27.96,6087720,-0.006873
1962-03,153.52,154.02,152.95,153.40,32.44,5687625,-0.006920
1962-04,131.59,132.38,129.80,130.56,27.62,13516935,-0.165792
1962-05,124.45,126.30,121.77,123.45,26.10,51659325,-0.140582
...,...,...,...,...,...,...,...
2016-08,3537.34,3553.86,3520.42,3535.17,2500.72,72912999,-0.002227
2016-09,3150.58,3176.76,3129.50,3150.43,2232.71,73154103,-0.000186
2016-10,3090.14,3108.85,3070.05,3086.77,2187.63,81930251,-0.033059


In [377]:
retM.index = pd.to_datetime(retM.index)
retM

,Open,High,Low,Close,Adj Close,Volume,Log Return
Month,,,,,,,
1962-01-01,155.95,156.87,154.77,155.37,32.86,9264945,-0.053346
1962-02-01,132.41,133.03,131.78,132.26,27.96,6087720,-0.006873
1962-03-01,153.52,154.02,152.95,153.40,32.44,5687625,-0.006920
1962-04-01,131.59,132.38,129.80,130.56,27.62,13516935,-0.165792
1962-05-01,124.45,126.30,121.77,123.45,26.10,51659325,-0.140582
...,...,...,...,...,...,...,...
2016-08-01,3537.34,3553.86,3520.42,3535.17,2500.72,72912999,-0.002227
2016-09-01,3150.58,3176.76,3129.50,3150.43,2232.71,73154103,-0.000186
2016-10-01,3090.14,3108.85,3070.05,3086.77,2187.63,81930251,-0.033059


In [378]:
ret_Jan=retM[retM.index.month==1]
ret_others=retM[retM.index.month!=1]
ret_Jan.head()

,Open,High,Low,Close,Adj Close,Volume,Log Return
Month,,,,,,,
1962-01-01,155.95,156.87,154.77,155.37,32.86,9264945,-0.053346
1963-01-01,114.69,115.66,114.16,115.03,24.35,29920830,0.082238
1964-01-01,149.72,151.65,148.85,149.92,31.73,23401635,0.063626
1965-01-01,137.27,138.14,136.85,137.76,29.24,11842812,0.096627
1966-01-01,165.67,166.70,164.65,165.58,35.17,13254912,-0.005935


In [379]:
ret_others.head()

,Open,High,Low,Close,Adj Close,Volume,Log Return
Month,,,,,,,
1962-02-01,132.41,133.03,131.78,132.26,27.96,6087720,-0.006873
1962-03-01,153.52,154.02,152.95,153.40,32.44,5687625,-0.006920
1962-04-01,131.59,132.38,129.80,130.56,27.62,13516935,-0.165792
1962-05-01,124.45,126.30,121.77,123.45,26.10,51659325,-0.140582
1962-06-01,91.94,93.99,89.73,91.38,19.34,71679765,-0.152580


In [380]:
print(sp.stats.ttest_ind(ret_Jan['Log Return'],ret_others['Log Return'])) 

Ttest_indResult(statistic=1.9748712016606744, pvalue=0.048700378446643794)


Since the T-statistic is 1.97 and the p-value is < 0.05, at a 5% significance level we can conclude there is a January effect for the IBM stock in the given time interval. That said, at any higher significance level this would not be conclusive.  A word
of caution: we should not generalize this result since it is based on just one stock. In terms of the weekday effect, we could apply the same procedure to test its existence. One end of chapter problems is designed to test the weekday effect based on the
same logic.

## 52-week high and low trading strategy

Some investors/researchers argue that we could adopt a 52-week high and low
trading strategy by taking a long position if today's price is close to the maximum
price achieved in the past 52 weeks and taking an opposite position if today's price is
close to its 52-week low. Let's randomly choose a day of 12/31/2016. The following
Python program presents this 52-week's range and today's position:

In [408]:
import numpy as np
from dateutil.relativedelta import relativedelta
#
ticker='IBM'
enddate= datetime.date(2023,12,31)
#
begdate=enddate-relativedelta(years=1)
today = '2023-10-02'

p =yf.download(ticker, start=begdate, end=enddate,rounding=True)
high= max(p['Adj Close'])
low=min(p['Adj Close'])
print(" Today, Price High Low, % from low ")


print(today, p[p.index == today]['Adj Close'].item(), high, low, round((p[p.index == today]['Adj Close'].item()-low)/(high-low)*100,2))

[*********************100%***********************]  1 of 1 completed
 Today, Price High Low, % from low 
2023-10-02 137.96 163.22 117.12 45.21


According to the 52-week high and low trading strategy, we have more incentive
to buy IBM's stock today. This example is just an illustration on how to make a
decision. There is nothing done to test whether this is a profitable trading strategy. If
a reader is interested in testing this 52-week high and low trading strategy, he/she
should use all stocks to form two portfolios. For more details, see George and
Huang (2004).

## Estimating Roll's spread

Liquidity is defined as how quickly we can dispose of our asset without losing its
intrinsic value. Usually, we use spread to represent liquidity. However, we need
high-frequency data to estimate spread. Later in the chapter, we show how to
estimate spread directly by using high-frequency data. To measure spread indirectly
based on daily observations, Roll (1984) shows that we can estimate it based on the
serial covariance in price changes, as follows...

Here, S is the Roll spread, Pt is the closing price of a stock on day, is Pt-Pt-1, and ,
t is the average share price in the estimation period. The following Python code
estimates Roll's spread for IBM, using one year's daily price data from Yahoo! Finance:

In [414]:
import scipy as sp
ticker='IBM'
begdate=datetime.date(2013,9,1)
enddate=datetime.date(2013,11,11)
data= yf.download(ticker, start = begdate, end = enddate, rounding = True)
p= data['Adj Close']
d= np.diff(p)
d

cov_=np.cov(d[:-1],d[1:])
if cov_[0,1]<0:
    print("Roll spread for ", ticker, 'is', round(2*np.sqrt(-cov_[0,1]),3))
else:
    print("Cov is positive for ",ticker, 'positive', round(cov_[0,1],3))

[*********************100%***********************]  1 of 1 completed
Roll spread for  IBM is 0.726


In [417]:
cov_

array([[ 2.58185828, -0.13189662],
       [-0.13189662,  2.57673469]])

The covariance between them is negative. When its value is positive, Roll's model
would fail. In a real world, it could occur for many cases. Usually, practitioners
adopt two approaches: when the spread is negative, we just ignore those cases or use
other methods to estimate spread. The second approach is to add a negative sign in
front of a positive covariance.

## Estimating Amihud's illiquidity

According to Amihud (2002), liquidity reflects the impact of order flow on price. Here, illiq(t) is the Amihud's illiquidity measure for month t, Ri is the daily return at day i, Pi is the closing price at i, and Vi is the daily dollar trading volume at i. Since
the illiquidity is the reciprocal of liquidity, the lower the illiquidity value, the higher
the liquidity of the underlying security. 

In the following code, we estimate Amihud's illiquidity for IBM based on trading
data in October 2013. The value is 1.21*10-11. It seems that this value is quite small.
Actually, the absolute value is not important; the relative value matters.

In [434]:
import numpy as np
import statsmodels.api as sm
begdate=datetime.date(2013,10,1)
enddate=datetime.date(2013,10,30)

stock='IBM'
stock2 = 'WMT'

def Illiq(ticker):
    data= yf.download(ticker, start =begdate, end =enddate, rounding = True)
    dollar_vol=np.array(data['Volume']*p) 
    ret=np.array((p[1:] - p[:-1])/p[1:])
    illiq= np.mean(np.divide(abs(ret),dollar_vol[1:])) 
    print("Aminud illiq for =",ticker,illiq)

Illiq(stock)
Illiq(stock2)

[*********************100%***********************]  1 of 1 completed
[-0.00768223 -0.00597172  0.00131556 -0.011533   -0.01843151  0.01433788
  0.01870139  0.00745945  0.00431816 -0.01250437  0.01106883 -0.06807056
 -0.00603977 -0.00532461  0.01208933  0.00459306  0.01135228 -0.00529535
  0.00282229  0.0261548 ]
Aminud illiq for = IBM 1.7908254141425208e-11
[*********************100%***********************]  1 of 1 completed
[-0.00768223 -0.00597172  0.00131556 -0.011533   -0.01843151  0.01433788
  0.01870139  0.00745945  0.00431816 -0.01250437  0.01106883 -0.06807056
 -0.00603977 -0.00532461  0.01208933  0.00459306  0.01135228 -0.00529535
  0.00282229  0.0261548 ]
Aminud illiq for = WMT 1.6794931630342896e-11


We can see here that IBM is more liquid than WMT, at least according to data from yahoo. WE are using close data, not adjusted close.

## Estimating Pastor and Stambaugh (2003) liquidity measure

Based on the methodology and empirical evidence in Campbell, Grossman, and
Wang (1993), Pastor and Stambaugh (2003) designed the following model to measure
individual stock's liquidity and the market liquidity.

Here, yt is the excess stock return, Rt-Rf , t, on day t, Rt is the return for the stock, Rf,t is
the risk-free rate, x1,t is the market return, and x2,t is the signed dollar trading volume:

pt is the stock price, and volume, t is the trading volume. The regression is run based
on daily data for each month. In other words, for each month, we get one β2 that is
defined as the liquidity measure for individual stock. The following code estimates
the liquidity for IBM. First, we download the IBM and S&P500 daily price data,
estimate their daily returns, and merge them as follows:

In [490]:
import pandas as pd
import statsmodels.api as sm
ticker='IBM'
begdate=datetime.date(2013,1,1)
enddate=datetime.date(2013,1,31) 

data =yf.download(ticker, start = begdate, end = enddate , rounding = True)
ret = data['Close'].pct_change()
dollar_vol= data['Close']*data['Volume']

tt=pd.DataFrame(ret)
tt2=pd.DataFrame(dollar_vol)

tt = tt.rename(columns={'Close':"Return"})
tt2 = tt2.rename(columns={0:"Dollar Volume"})

tt3=pd.merge(tt,tt2,left_index=True,right_index=True) 
final=pd.merge(tt3,f3factor,left_index=True,right_index=True)
final = final.dropna()



[*********************100%***********************]  1 of 1 completed


In [494]:
final.tail()

,Return,Dollar Volume,Mkt-RF,SMB,HML,RF
Date,,,,,,
2013-01-24,-0.001482,9.090956e+08,0.09,0.29,0.04,0.0
2013-01-25,0.002712,6.884876e+08,0.59,-0.14,-0.32,0.0
2013-01-28,-0.000204,5.786674e+08,-0.14,0.41,0.15,0.0
2013-01-29,-0.005053,7.375151e+08,0.36,-0.44,0.31,0.0
2013-01-30,-0.001847,6.102349e+08,-0.38,-0.84,0.16,0.0


In [491]:
y= final['Return'][1:] - final['RF'][1:]
x1= np.array(final['Mkt-RF'][:-1])

x1b= np.array(final['Return'][:-1]-final['RF'][:-1])
x1c = np.array(final['Dollar Volume'][:-1])
x2 = np.sign(x1b*x1c)
x3 = [x1,x2]
n = np.size(x3)

x = np.reshape(x3, [int(n/2),2])
x = sm.add_constant(x)
results = sm.OLS(y,x).fit()
print(results.params)



const    0.004176
x1       0.005786
x2      -0.006879
dtype: float64


In the previous program, y is IBM's excess return at time t+1, x1 is the market excess
return at time t, and x2 is the signed dollar trading volume at time t. The coefficient
before x2 is Pastor and Stambaugh's liquidity measure. The corresponding output is given above.

## Fama-MacBeth regression
First, let's look at the OLS regression by using the pandas.ols function as follows:

In [501]:
import numpy as np
import pandas as pd
n = 252
np.random.seed(12345)
begdate=datetime.date(2013, 1, 2)
dateRange = pd.date_range(begdate, periods=n)
x0= pd.DataFrame(np.random.randn(n, 1),columns=['ret'],index=dateRange)
y0= pd.Series(np.random.randn(n), index=dateRange)
print (sm.OLS(y0, x0).fit().params)

ret   -0.009658
dtype: float64


Just ignore this section - it appears the code is outdated and I'm uncertain as to what will replace it.

## Durbin Watson Test Statistic

Durbin-Watson statistic is related auto-correlation. After we run a regression, the
error term should have no correlation, with a mean zero. 

Here, et is the error term at time t, T is the total number of error term. The
Durbin-Watson statistic tests the null hypothesis that the residuals from an ordinary
least-squares regression are not auto-correlated against the alternative that the
residuals follow an AR1 process. The Durbin-Watson statistic ranges in value from
0 to 4. A value near 2 indicates non-autocorrelation; a value toward 0 indicates
positive autocorrelation; a value toward 4 indicates negative autocorrelation.

The following Python program runs a CAPM first by using daily data for IBM. The
S&P500 is used as the index. The time period is from 1/1/2012 to 12/31/2016, a
5-year window. The risk-free rate is ignored in this case. For the residual from the
regression, a Durbin-Watson test is run to test its autocorrelation:


In [512]:
import statsmodels.stats.stattools as tools 

begdate=datetime.date(2012,1,1)
enddate=datetime.date(2016,12,31)

def dailyRet(ticker,begdate,enddate):
    p =yf.download(ticker, start = begdate, end =enddate, rounding = True)
    return p['Close'].pct_change().dropna()

retIBM=dailyRet('IBM',begdate,enddate)
retMkt=dailyRet('^GSPC',begdate,enddate)

df = pd.DataFrame({"Y":retIBM, "X": retMkt})

X = sm.add_constant(df['X'])
result = sm.OLS(df['Y'], X).fit()
print(result.params)
residuals= result.resid
print("Durbin Watson")
print(tools.durbin_watson(residuals))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
const   -0.000446
X        0.883940
dtype: float64
Durbin Watson
1.816250797254747


Based on these results, since Durbin Watson statistic < 2, the autocorrelation might be zero for the residuals from the CAPM for IBM. A more definitive answer can be provided by print(result.summary()) as seen below.

In [513]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     730.9
Date:                Tue, 20 Feb 2024   Prob (F-statistic):          3.13e-127
Time:                        14:14:52   Log-Likelihood:                 4088.2
No. Observations:                1257   AIC:                            -8172.
Df Residuals:                    1255   BIC:                            -8162.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004      0.000     -1.684      0.0